## Visualize Vegetation Indices in Timeseries

*Author: Alessandro Jopshua Pierro*

*Affiliation: Agrscope*

*Date: December 2023*

## Overview

This Python notebook is focused on visualizing various vegetation indices, specifically NDVI, NDRE, and EVI, over time using Sentinel-2 satellite imagery. The indices are vital for assessing the health and growth patterns of agricultural crops. This visualization helps in understanding the temporal changes in crop conditions and the effectiveness of agricultural practices.
## Objective
The primary goal is to provide a clear visual representation of changes in vegetation indices over time for specific agricultural parcels. This helps in identifying patterns, anomalies, and trends in crop growth and health, facilitating informed decision-making in agriculture management.
## Methodology
The process involves preprocessing Sentinel-2 imagery, extracting essential spectral data, and utilizing shapefiles for precise geographic targeting. 
## Key functions
preprocess_sentinel2_scenes: Handles the resampling and masking of clouds, shadows, and snow in Sentinel-2 scenes.

extract_s2_data: Extracts and filters Sentinel-2 data based on cloud cover and time period for given parcels.

save_scene_as_geotiff: Saves processed Sentinel-2 scenes as GeoTIFF files for further analysis or archiving.

plot_scenes: Generates plots for each vegetation index per scene, offering a visual representation of the data.

process_shapefiles_in_subfolders: Automates the processing of shapefiles in subdirectories, facilitating batch processing of data.
## Execution
Run the code to process shapefiles in specified subfolders within a base directory. The script will iterate through each shapefile, extract relevant Sentinel-2 data, and generate plots for the NDVI, NDRE, and EVI indices for each parcel.

## Import Libaries

In [7]:
import os
import glob
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime
import warnings

from datetime import datetime
from eodal.core.scene import SceneCollection
from eodal.core.sensors.sentinel2 import Sentinel2
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from eodal.config import get_settings

# Set environment variable to enable working with STAC from Planetary Computer
settings = get_settings()
settings.USE_STAC = True

# Define eodal functions and plotting

In [8]:
def preprocess_sentinel2_scenes(
    ds: Sentinel2,
    target_resolution: int,
) -> Sentinel2:
    """
    Resample Sentinel-2 scenes and mask clouds, shadows, and snow
    based on the Scene Classification Layer (SCL).

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
    interfacing libraries!

    :param target_resolution:
        spatial target resolution to resample all bands to.
    :returns:
        resampled, cloud-masked Sentinel-2 scene.
    """
    # resample scene
    ds.resample(inplace=True, target_resolution=target_resolution)
    # mask clouds, shadows, and snow
    ds.mask_clouds_and_shadows(inplace=True, cloud_classes=[3, 8, 9, 10, 11])
    return ds

def extract_s2_data(
        parcel: gpd.GeoDataFrame,
        time_start: datetime,
        time_end: datetime,
        scene_cloud_cover_threshold: float = 20,
        feature_cloud_cover_threshold: float = 20,
        spatial_resolution: int = 10
    ) -> SceneCollection:
    """
    Extracts Sentinel-2 data from the STAC SAT archive for a given area and time period.
    Scenes that are too cloudy or contain nodata (blackfill), only, are discarded.

    The processing level of the Sentinel-2 data is L2A (surface reflectance factors).

    :param parcel:
        field parcel geometry (defines the spatial extent to extract)
    :param time_start:
        start of the time period to extract
    :param end_time:
        end of the time period to extract
    :param scene_cloud_cover_threshold:
        scene-wide cloudy pixel percentage (from Sentinel-2 metadata) to filter out scenes
        with too high cloud coverage values [0-100%]
    :param feature_cloud_cover_threshold:
        cloudy pixel percentage [0-100%] on the parcel level. Only if the parcel has a
        lower percentual share of cloudy pixels (based on the scene classification layer) than
        the threshold specified, the Sentinel-2 observation is kept
    :param spatial_resolution:
        spatial resolution of the Sentinel-2 data in meters (Def: 10m)
    :param resampling_method:
        spatial resampling method for those Sentinel-2 bands not available in the target
        resolution. Nearest Neighbor by default
    :returns:
        dictionary with the list of scenes for the field parcel (`feature_scenes`), the
        DataFrame of (un)used scenes and the reason for not using plus some basic scene
        metadata (`scene_properties`)
    """
    # setup the metadata filters (cloud cover and processing level)
    metadata_filters = [
        Filter('cloudy_pixel_percentage','<', scene_cloud_cover_threshold),
        Filter('processing_level', '==', 'Level-2A')
    ]
    # setup the spatial feature for extracting data
    feature = Feature.from_geoseries(parcel.geometry)
    
    # set up mapping configs
    mapper_configs = MapperConfigs(
        collection='sentinel2-msi',
        time_start=time_start,
        time_end=time_end,
        feature=feature,
        metadata_filters=metadata_filters
    )

    # get a new mapper instance. Set sensor to `sentinel2`
    mapper = Mapper(mapper_configs)

    # query the STAC (looks for available scenes in the selected spatio-temporal range)
    mapper.query_scenes()

    # get observations (loads the actual Sentinel2 scenes)
    # the data is extract for the extent of the parcel
    scene_kwargs = {
        'scene_constructor': Sentinel2.from_safe,            # this tells the mapper how to read and load the data (i.e., Sentinel-2 scenes)
        'scene_constructor_kwargs': {},                      # here you could specify which bands to read
        'scene_modifier': preprocess_sentinel2_scenes,       # this tells the mapper about (optional) pre-processing of the loaded scenes (must be a callable)
        'scene_modifier_kwargs': {'target_resolution': 10}   # here, you have to specify the value of the arguments the `scene_modifier` requires
    }
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    # loop over available Sentinel-2 scenes stored in mapper.data as a EOdal SceneCollection and check
    # for no-data. These scenes will be removed from the SceneCollection
    scenes_to_del = []
    mapper.metadata['scene_used'] = 'yes'
    for scene_id, scene in mapper.data:

        # check if scene is blackfilled (nodata); if yes continue
        if scene.is_blackfilled:
            scenes_to_del.append(scene_id)
            mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [blackfill]'
            continue

        # check cloud coverage (including shadows and snow) of the field parcel
        feature_cloud_cover = scene.get_cloudy_pixel_percentage(cloud_classes=[3, 8, 9, 10, 11])

        # if the scene is too cloudy, we skip it
        if feature_cloud_cover > feature_cloud_cover_threshold:
            scenes_to_del.append(scene_id)
            mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [clouds]'
            continue

        # calculate the NDVI, NDRE and EVI
        scene.calc_si('NDVI', inplace=True)
        scene.calc_si('NDRE', inplace=True)
        scene.calc_si('EVI', inplace=True)

    # delete scenes too cloudy or containing only no-data
    for scene_id in scenes_to_del:
        del mapper.data[scene_id]
    
    return mapper

def save_scene_as_geotiff(scene, shapefile_name, metadata, scene_id, res):
    indices = ['ndvi', 'ndre', 'evi']
    base_paths = {
        'ndvi': "C:\\Users\\aless\\Desktop\\Agroscope\\FerN\\results\\parcel_visualisation\\field_indices\\raster\\ndvi",
        'ndre': "C:\\Users\\aless\\Desktop\\Agroscope\\FerN\\results\\parcel_visualisation\\field_indices\\raster\\ndre",
        'evi': "C:\\Users\\aless\\Desktop\\Agroscope\\FerN\\results\\parcel_visualisation\\field_indices\\raster\\evi"
    }

    # Get the list of sensing dates marked with 'yes' in 'scene_used' column
    sensing_dates_with_scene_used_list = metadata[metadata['scene_used'] == 'yes']['sensing_date'].tolist()

    for idx, (scene_id, scene) in enumerate(res):
        sensing_date_string = sensing_dates_with_scene_used_list[idx].strftime('%Y-%m-%d') if idx < len(sensing_dates_with_scene_used_list) else "No Date"
        
        # Create subfolders for each index and shapefile name
        for index in indices:
            subfolder_path = os.path.join(base_paths[index], shapefile_name)
            ensure_directory_exists(subfolder_path)
            
            file_path = os.path.join(subfolder_path, f"{shapefile_name}_{index}_{sensing_date_string}.tiff")
            scene.to_rasterio(file_path, band_selection=[index], as_cog=True)

In [9]:
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def suppress_warnings():
    warnings.filterwarnings('ignore')
    
def plot_scenes(res: SceneCollection, shapefile_name: str, metadata, save_path: str, scene_id) -> None:
    f, axes = plt.subplots(ncols=len(res), nrows=3, figsize=(26, 14))  # Only 3 rows
    # Check if axes is one-dimensional and convert to two-dimensional if necessary
    
    # Get list of sensing dates marked with 'yes' in 'scene_used' column
    sensing_dates_with_scene_used_list = metadata[metadata['scene_used'] == 'yes']['sensing_date'].tolist()
    
    if len(res) == 1:
        axes = np.array([[ax] for ax in axes]).reshape(3, -1)
    
    # Set the main title with the shapefile name
    f.suptitle(shapefile_name, fontsize=16)
    
    for idx, (scene_id, scene) in enumerate(res):
        # Get the sensing date for the current column and format it as a string
        sensing_date_string = sensing_dates_with_scene_used_list[idx].strftime('%Y-%m-%d') if idx < len(sensing_dates_with_scene_used_list) else "No Date"
        
        # Sensing date titles
        y_position = 0.92  # Decrease this value to move the text lower
        plt.figtext(0.5/len(res) + (idx * (1/len(res))), y_position, sensing_date_string, ha='center', fontsize=8)
        
        # Plot NDVI on the first row
        scene.plot_band(
            'NDVI',
            colormap='summer',
            colorbar_label='NDVI',
            vmin=0,
            vmax=0.8,
            ax=axes[0, idx]
        )
        axes[0, idx].set_title(f"NDVI")

        # Plot NDRE on the second row
        scene.plot_band(
            'NDRE',
            colormap='winter',
            colorbar_label='NDRE',
            vmin=0,
            vmax=1.0,
            ax=axes[1, idx]
        )
        axes[1, idx].set_title(f"NDRE")

        # Plot EVI on the third row
        scene.plot_band(
            'EVI',
            colormap='autumn',
            colorbar_label='EVI',
            vmin=0,
            vmax=1.0,
            ax=axes[2, idx]
        )
        axes[2, idx].set_title(f"EVI")

        if idx > 0:
            for jdx in range(3):
                axes[jdx, idx].get_yaxis().set_ticks([])
                axes[jdx, idx].set_ylabel('')
                

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the rect to make space for the main title

    # Save plots as PNG and PDF
    png_file = os.path.join(save_path, 'png', f"{shapefile_name}.png")
    pdf_file = os.path.join(save_path, 'pdf', f"{shapefile_name}.pdf")
    plt.savefig(png_file, format='png')
    plt.savefig(pdf_file, format='pdf')
    plt.close()

In [10]:
def process_shapefiles_in_subfolders(base_folder: str, time_start: datetime, time_end: datetime):
    suppress_warnings()
    # Define the paths for saving the plots
    save_path = "C:\\Users\\aless\\Desktop\\Agroscope\\FerN\\results\\parcel_visualisation\\field_indices"
    # Iterate over all subdirectories in the base_folder
    for subdir in os.listdir(base_folder):
        subdir_path = os.path.join(base_folder, subdir)
        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Use glob to find all .shp files in the current subdirectory
            shapefiles = glob.glob(os.path.join(subdir_path, "*.shp"))
            for shp_file in shapefiles:
                # Extract the name without the '.shp' extension
                shapefile_name = os.path.splitext(os.path.basename(shp_file))[0]
                print(f"Processing {shapefile_name}")
                # Read the shapefile
                parcel = gpd.read_file(shp_file)
                # Extract Sentinel-2 data for the shapefile
                res = extract_s2_data(
                    parcel=parcel,
                    time_start=time_start,
                    time_end=time_end
                )
                metadata = res.metadata[['product_uri', 'sensing_date', 'scene_used']]
                for scene_id, scene in res.data:
                   save_scene_as_geotiff(scene,shapefile_name,metadata,scene_id,res.data)   
                # Plot the scenes if data is available
                if res.data:
                    plot_scenes(res.data,shapefile_name,metadata,save_path,scene_id)
                else:
                    print(f"No data available for {shapefile_name}")


# Execute the functions 

In [11]:
# Define the base folder path and the time period for data extraction
base_folder_path = "C:\\Users\\aless\\Desktop\\Agroscope\\FerN\\field_information\\Shapefiles_Fields"
time_start = datetime(2023, 3, 1)
time_end = datetime(2023, 5, 31)

# Call the function to process all shapefiles in subfolders
process_shapefiles_in_subfolders(base_folder_path, time_start, time_end)

Processing Blé Sorens


2023-12-08 14:19:17,983 eodal        INFO     Starting extraction of sentinel2 scenes
2023-12-08 14:19:29,196 eodal        INFO     Finished extraction of sentinel2 scenes


Processing Bellechasse_Colza


2023-12-08 14:19:34,156 eodal        INFO     Starting extraction of sentinel2 scenes
2023-12-08 14:19:38,780 eodal        INFO     Finished extraction of sentinel2 scenes


Processing Bellechasse Epeautre


...

2023-12-08 14:20:44,579 eodal        INFO     Starting extraction of sentinel2 scenes
2023-12-08 14:20:51,332 eodal        INFO     Finished extraction of sentinel2 scenes


Processing Champs_Payerne


2023-12-08 14:20:55,881 eodal        INFO     Starting extraction of sentinel2 scenes
2023-12-08 14:21:17,665 eodal        INFO     Finished extraction of sentinel2 scenes


Processing Champ Wuennewil


2023-12-08 14:21:24,807 eodal        INFO     Starting extraction of sentinel2 scenes
2023-12-08 14:21:35,897 eodal        INFO     Finished extraction of sentinel2 scenes
